In [28]:
from dataclasses import dataclass,field

@dataclass(kw_only=True)
class UID:
    index: int = field(default=0,init=False,repr=False)
    UID: str = field(default=None,init=False,repr=False)
    UID_name: str = field(default=None,init=False,repr=False)

    def setUID(self,UID_name=None):
        if self.UID_name is None:
            self.UID_name = UID_name

        if self.UID_name is None or not hasattr(self,self.UID_name):
            print('Field does not exist in object, Cannot set UID')
        
        self.updateUID()


    def updateUID(self):
        self.index+=1
        if self.UID is None:
            ID = getattr(self,self.UID_name)
        else:
            ID = self.UID.rsplit('_',1)[0]
        self.UID = f"{ID}_{self.index}"
        setattr(self,self.UID_name,self.UID)

@dataclass(kw_only=True)
class dummy(UID):
    ID: str = None
    UID_name: str = 'ID'
    d2: str = field(init=False)
    AD: str = field(init=False)

    def __post_init__(self):
        # self.setUID('ID')
        # self.updateUID()
        print(self.__dataclass_fields__.keys(),self.__annotations__.keys(),self.__dict__.keys())
        # pass

d = dummy(ID='1')
d.setUID()
d.UID

dict_keys(['index', 'UID', 'UID_name', 'ID', 'd2', 'AD']) dict_keys(['ID', 'UID_name', 'd2', 'AD']) dict_keys(['UID_name', 'ID'])


'1_1'

In [45]:
import inspect
from dataclasses import dataclass,field

@dataclass(kw_only=True)
class dummy():
    ID: str = '1'
    AD: str


    @classmethod
    def from_dict(cls, env):  
        # Source - https://stackoverflow.com/a/55096964
        # Posted by Arne, modified by community. See post 'Timeline' for change history
        # Retrieved 2025-11-21, License - CC BY-SA 4.0    
        print(env.items())
        print(inspect.signature(cls).parameters)
        print(cls.__dataclass_fields__)
        return cls(**{
            k: v for k, v in env.items() 
            if k in inspect.signature(cls).parameters
        })


# d = dummy(ID='a')
b = dummy.from_dict({'ID':'a','AD':'x'})
print(b.__dict__)

dict_items([('ID', 'a'), ('AD', 'x')])
OrderedDict([('ID', <Parameter "ID: str = '1'">), ('AD', <Parameter "AD: str">)])
{'ID': Field(name='ID',type=<class 'str'>,default='1',default_factory=<dataclasses._MISSING_TYPE object at 0x000001DF8DD48820>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=True,_field_type=_FIELD), 'AD': Field(name='AD',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x000001DF8DD48820>,default_factory=<dataclasses._MISSING_TYPE object at 0x000001DF8DD48820>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=True,_field_type=_FIELD)}
{'ID': 'a', 'AD': 'x'}


In [ ]:
from dataclasses import dataclass
@dataclass
class test: 

    a: str = 'a'

@dataclass
class test2(test):

    a: str
    b: str = 'b'

print(test2().__dict__)


{'a': 'a', 'b': 'b'}


In [ ]:
from dataclasses import dataclass, field
import numpy as np

@dataclass(kw_only=True)
class EC:
    measurementHeight: float = field(default = None, metadata = {'description': 'Measurement height (Zm) in meters, required for Sonics, optional otherwise'})
    northOffset: float = field(default = None, metadata = {'description': 'Offset from North in degrees (clockwise) of main sonic'})


    def __post_init__(self):
        pass

    
    def geographicSeparation(self,xSeparation,ySeparation):
        # Convert to radians
        # **Note**: north offset is relative to geographic (meteorologic) north, while x,y offsets are in cartesian coordinates.  To perform the coordinate rotation properly theta must be converted to cartesian coordinate (positive is counter-clockwise from the x axis)
        theta = np.deg2rad(270-self.northOffset)
        # Calculate counter-clockwise rotation matrix
        R = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
        # Evaluate rotation matrix.
        v = np.array([[xSeparation,ySeparation]])
        Rv = (R*v)
        Rv = Rv.sum(axis=1).round(3)
        northwardSeparation = float(Rv[1])
        eastwardSeparation = float(Rv[0])

        return(northwardSeparation,eastwardSeparation)
    
    def cartesianSeparation(self,northwardSeparation,eastwardSeparation):
        # Get the inverse of geographic separation
        # If xy not provided, calculate, if provided along with north/south check values make sense
        theta = np.deg2rad(self.northOffset-270)
        R = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
        v = np.array([[eastwardSeparation,northwardSeparation]])
        Rv = (R*v)
        Rv = Rv.sum(axis=1).round(3)
        xSeparation = float(Rv[0])
        ySeparation = float(Rv[1])
        return (xSeparation,ySeparation)